In [5]:
import pandas as pd
import numpy as np
import random

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split

from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import lightgbm as lgb
import os

In [2]:
train = pd.read_csv("../03-data-creation/train_processed_2.csv")
test = pd.read_csv("../03-data-creation/test_processed_2.csv")

# Drop unwanted column
train = train.drop("Unnamed: 0", axis = 1)
test = test.drop("Unnamed: 0", axis = 1)

# We won't touch test until predicting for submission

# We don't want the ID to be used in the model so preserve
train_id = train['id']
test_id = test['id']

train = train.drop('id', axis = 1)
test = test.drop('id', axis = 1)

# Split X and y
X = train.loc[:, train.columns != "satisfied"]
y = train.satisfied

In [3]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 3096)

In [6]:
# Instantiate model default params
rf = RandomForestClassifier(random_state = 3096)

# Train the model on training data
rf.fit(X_train, y_train);

In [7]:
# Use the forest's predict method on the test data
expected_y = y_test
predicted_y = rf.predict(X_test)


# Summarize model fit
print(); print(metrics.classification_report(expected_y, predicted_y))
print(); print(metrics.confusion_matrix(expected_y, predicted_y))
print(); print(metrics.roc_auc_score(expected_y, predicted_y))


              precision    recall  f1-score   support

           0       0.81      0.78      0.80      2907
           1       0.80      0.83      0.82      3109

    accuracy                           0.81      6016
   macro avg       0.81      0.81      0.81      6016
weighted avg       0.81      0.81      0.81      6016


[[2275  632]
 [ 517 2592]]

0.8081511636102473


Hyperparameter tuning:

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters

# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=10, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 64.8min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 110.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 118.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 179.8min


In [ ]:
rf_random.best_params_

In [ ]:
# Predict
preds = rf_random.predict(X_test)

# AUC calculation
best_random = rf_random.best_estimator_
random_accuracy = metrics.roc_auc_score(best_random, X_test, y_test)